## Imports

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.signal import find_peaks
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten, LSTM
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler


## Load Datasets


In [3]:
# Load datasets
train_data = pd.read_csv('TrainData_B.csv')
aggregated_load = pd.read_csv('AggregatedLoad_B.csv')
test_data = pd.read_csv('TestData_A.csv')
template = pd.read_csv('DisaggregatedLoad_Template.csv')


## Rename Columns for Convenience


In [4]:
# Rename columns for convenience
train_data.columns = ['index', 'aggregated_load'] + [f'appliance_{i}' for i in range(1, 22)]
aggregated_load.columns = ['index', 'aggregated_load']
test_data.columns = ['index', 'aggregated_load'] + [f'appliance_{i}' for i in range(1, 22)]
template.columns = ['index'] + [f'appliance_{i}_pred' for i in range(1, 22)]


## Add Rolling Mean Feature


In [5]:
# Add rolling mean feature to train, test, and aggregated_load datasets
window_size = 10

train_data['agg_load_mean'] = train_data['aggregated_load'].rolling(window=window_size, min_periods=1).mean()
test_data['agg_load_mean'] = test_data['aggregated_load'].rolling(window=window_size, min_periods=1).mean()
aggregated_load['agg_load_mean'] = aggregated_load['aggregated_load'].rolling(window=window_size, min_periods=1).mean()


## Add Rolling Standard Deviation Feature


In [6]:
# Add rolling std feature to train, test, and aggregated_load datasets
train_data['agg_load_std'] = train_data['aggregated_load'].rolling(window=window_size, min_periods=1).std()
test_data['agg_load_std'] = test_data['aggregated_load'].rolling(window=window_size, min_periods=1).std()
aggregated_load['agg_load_std'] = aggregated_load['aggregated_load'].rolling(window=window_size, min_periods=1).std()


## Calculate Frequency Feature


In [7]:
def calculate_frequency(data, window_size):
    # Find local maxima and minima
    peaks, _ = find_peaks(data)
    troughs, _ = find_peaks(-data)
    
    # Combine and sort indices of peaks and troughs
    extrema = np.sort(np.concatenate([peaks, troughs]))
    
    # Calculate frequency of extrema within the window
    frequency = np.zeros_like(data)
    for i in range(len(data)):
        start = max(0, i - window_size)
        end = i
        frequency[i] = np.sum((extrema >= start) & (extrema < end))
    
    return frequency

# Add frequency feature to train, test, and aggregated_load datasets
train_data['agg_load_freq'] = calculate_frequency(train_data['aggregated_load'].values, window_size)
test_data['agg_load_freq'] = calculate_frequency(test_data['aggregated_load'].values, window_size)
aggregated_load['agg_load_freq'] = calculate_frequency(aggregated_load['aggregated_load'].values, window_size)


## Add Lagged Aggregated Load Feature


In [8]:
# Add lagged aggregated load feature
train_data['lagged_agg_load'] = train_data['aggregated_load'].shift(1)
test_data['lagged_agg_load'] = test_data['aggregated_load'].shift(1)
aggregated_load['lagged_agg_load'] = aggregated_load['aggregated_load'].shift(1)

# Drop the first row as it will have NaN values due to shifting
train_data.dropna(inplace=True)
test_data.dropna(inplace=True)
aggregated_load.dropna(inplace=True)


## Normalize Data


In [9]:
# Normalize the data for aggregated load, its mean, frequency, and lagged aggregated load separately
scaler_agg = StandardScaler()
scaler_agg_mean = StandardScaler()
scaler_agg_std = StandardScaler()
scaler_agg_freq = StandardScaler()
scaler_agg_lagged = StandardScaler()
scaler_appliances = StandardScaler()

# Fit scaler on the aggregated load, its mean, frequency, and lagged aggregated load from training data
train_agg_normalized = scaler_agg.fit_transform(train_data[['aggregated_load']])
train_agg_std_normalized = scaler_agg_std.fit_transform(train_data[['agg_load_std']])
train_agg_mean_normalized = scaler_agg_mean.fit_transform(train_data[['agg_load_mean']])
train_agg_freq_normalized = scaler_agg_freq.fit_transform(train_data[['agg_load_freq']])
train_lagged_normalized = scaler_agg_lagged.fit_transform(train_data[['lagged_agg_load']])

# Fit scaler on the appliance loads from training data
train_appliances_normalized = scaler_appliances.fit_transform(train_data.iloc[:, 2:-4])  # Exclude agg_load_std, agg_load_mean, agg_load_freq, and lagged_agg_load

# Concatenate the normalized features
train_features_normalized = np.concatenate([
    train_agg_normalized, train_agg_std_normalized, train_agg_mean_normalized, train_agg_freq_normalized, train_lagged_normalized
], axis=1)
test_features_normalized = np.concatenate([
    scaler_agg.transform(test_data[['aggregated_load']]),
    scaler_agg_std.transform(test_data[['agg_load_std']]),
    scaler_agg_mean.transform(test_data[['agg_load_mean']]),
    scaler_agg_freq.transform(test_data[['agg_load_freq']]),
    scaler_agg_lagged.transform(test_data[['lagged_agg_load']])
], axis=1)
aggregated_features_normalized = np.concatenate([
    scaler_agg.transform(aggregated_load[['aggregated_load']]),
    scaler_agg_std.transform(aggregated_load[['agg_load_std']]),
    scaler_agg_mean.transform(aggregated_load[['agg_load_mean']]),
    scaler_agg_freq.transform(aggregated_load[['agg_load_freq']]),
    scaler_agg_lagged.transform(aggregated_load[['lagged_agg_load']])
], axis=1)


## Split Data into Training and Test Sets


In [10]:
# Split data into training and test sets (80% training, 20% test)
X_train, X_test, y_train, y_test = train_test_split(train_features_normalized, train_appliances_normalized, test_size=0.2, random_state=42)

# Reshape the training and test data
X_train = X_train.reshape(-1, 1, 5)  # Include 4 features: aggregated load, its mean, its frequency, and lagged aggregated load
X_test = X_test.reshape(-1, 1, 5)
X_aggregated = aggregated_features_normalized.reshape(-1, 1, 5)


## CNN Model

In [11]:
# CNN Model
cnn_model = Sequential()
cnn_model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(1, 5)))  # Update input_shape to (1, 5)
cnn_model.add(Flatten())
cnn_model.add(Dense(21, activation='linear'))

cnn_model.compile(optimizer=Adam(), loss='mean_squared_error')
cnn_model.summary()

# CNN Training
cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Predicting with CNN
y_pred_cnn_test = cnn_model.predict(X_test)
y_pred_cnn_full = cnn_model.predict(X_aggregated)


C:\Users\jihad\PycharmProjects\energyinfo\venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 1, 64)          │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 21)             │         1,365 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,749 (6.83 KB)

 Trainable params: 1,749 (6.83 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.8866 - val_loss: 0.8538
Epoch 2/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.8396 - val_loss: 0.8255
Epoch 3/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.8367 - val_loss: 0.8095
Epoch 4/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.7809 - val_loss: 0.7985
Epoch 5/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.7725 - val_loss: 0.7865
Epoch 6/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 2s 975us/step - loss: 0.7567 - val_loss: 0.7849
Epoch 7/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 2s 966us/step - loss: 0.7753 - val_loss: 0.7729
Epoch 8/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 2s 912us/step - loss: 0.7287 - val_loss: 0.7710
Epoch 9/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.7804 - val_loss: 0.7619
Epoch 10/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.7551 - val_loss: 0.7627
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 498us/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 493us/step


## LSTM Model

In [12]:
# LSTM Model
lstm_model = Sequential()
lstm_model.add(LSTM(50, activation='relu', input_shape=(1, 5)))  # Update input_shape to (1, 5)
lstm_model.add(Dense(21))

lstm_model.compile(optimizer=Adam(), loss='mean_squared_error')
lstm_model.summary()

# LSTM Training
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Predicting with LSTM
y_pred_lstm_test = lstm_model.predict(X_test)
y_pred_lstm_full = lstm_model.predict(X_aggregated)


C:\Users\jihad\PycharmProjects\energyinfo\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 50)             │        11,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 21)             │         1,071 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,271 (47.93 KB)

 Trainable params: 12,271 (47.93 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 3s 783us/step - loss: 0.8891 - val_loss: 0.8436
Epoch 2/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 2s 733us/step - loss: 0.8327 - val_loss: 0.8124
Epoch 3/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 2s 749us/step - loss: 0.7579 - val_loss: 0.7970
Epoch 4/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 2s 756us/step - loss: 0.7686 - val_loss: 0.7861
Epoch 5/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 2s 738us/step - loss: 0.7400 - val_loss: 0.7771
Epoch 6/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 2s 878us/step - loss: 0.7366 - val_loss: 0.7662
Epoch 7/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 2s 910us/step - loss: 0.7173 - val_loss: 0.7603
Epoch 8/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 2s 889us/step - loss: 0.7326 - val_loss: 0.7520
Epoch 9/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 2s 901us/step - loss: 0.7028 - val_loss: 0.7452
Epoch 10/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 2s 946us/step - loss: 0.7346 - val_loss: 0.7393
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 470us/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 551us/step


## Random Forest Model

In [13]:
# Random Forest Model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train.reshape(-1, 5), y_train)  # Reshape X_train for RandomForest

# Predicting with Random Forest
y_pred_rf_test = rf_model.predict(X_test.reshape(-1, 5))
y_pred_rf_full = rf_model.predict(X_aggregated.reshape(-1, 5))


## Calculate RMSE

In [15]:
# Function to calculate RMSE
def calculate_rmse(true_values, pred_values):
    true_values_scaled = true_values / true_values.max(axis=0)
    pred_values_scaled = pred_values / true_values.max(axis=0)
    rmse = np.sqrt(mean_squared_error(true_values_scaled, pred_values_scaled, multioutput='raw_values'))
    return rmse

# Calculate RMSE for each appliance
rmse_cnn = calculate_rmse(y_test, y_pred_cnn_test)
rmse_lstm = calculate_rmse(y_test, y_pred_lstm_test)
rmse_rf = calculate_rmse(y_test, y_pred_rf_test)

# Combine RMSE into a DataFrame for comparison
rmse_df = pd.DataFrame({
    'appliance': [f'appliance_{i}' for i in range(1, 22)],
    'rmse_cnn': rmse_cnn,
    'rmse_lstm': rmse_lstm,
    'rmse_rf': rmse_rf
})

# Determine the best model for each appliance
best_models = rmse_df[['rmse_cnn', 'rmse_lstm', 'rmse_rf']].idxmin(axis=1)
best_models = best_models.replace({'rmse_cnn': 'CNN', 'rmse_lstm': 'LSTM', 'rmse_rf': 'RF'})
rmse_df['best_model'] = best_models



## Hybrid Model Predictions


In [16]:
# Initialize the hybrid predictions array for test data
hybrid_predictions_test = np.zeros_like(y_pred_cnn_test)

# Assign the best model's predictions to the hybrid model for test data
for i in range(21):
    if rmse_df.loc[i, 'best_model'] == 'CNN':
        hybrid_predictions_test[:, i] = y_pred_cnn_test[:, i]
    elif rmse_df.loc[i, 'best_model'] == 'LSTM':
        hybrid_predictions_test[:, i] = y_pred_lstm_test[:, i]
    else:
        hybrid_predictions_test[:, i] = y_pred_rf_test[:, i]

# Calculate and show summed RMSE for the hybrid model on test data
rmse_hybrid_test = calculate_rmse(y_test, hybrid_predictions_test)
print("Hybrid Model RMSE Sum on Test Data:", rmse_hybrid_test)

# Initialize the hybrid predictions array for AggregatedLoad_A
hybrid_predictions_full = np.zeros_like(y_pred_cnn_full)

# Assign the best model's predictions to the hybrid model for AggregatedLoad_A
for i in range(21):
    if rmse_df.loc[i, 'best_model'] == 'CNN':
        hybrid_predictions_full[:, i] = y_pred_cnn_full[:, i]
    elif rmse_df.loc[i, 'best_model'] == 'LSTM':
        hybrid_predictions_full[:, i] = y_pred_lstm_full[:, i]
    else:
        hybrid_predictions_full[:, i] = y_pred_rf_full[:, i]


Hybrid Model RMSE Sum on Test Data: [0.05539353 0.01274248 0.01774435 0.01999191 0.00981476 0.04790708
 0.0160024  0.02248768 0.00939595 0.08089377 0.00724826 0.07088784
 0.01125993 0.03290468 0.04795626 0.06732701 0.03356306 0.08017213
 0.01822416 0.01408661 0.01422649]


## Save Predictions to CSV


In [17]:
# Save predictions to CSV
def save_predictions(predictions, filename):
    predictions_scaled = scaler_appliances.inverse_transform(predictions)
    template_copy = template.copy()
    if len(predictions_scaled) < len(template_copy):
        # Extend predictions_scaled to match the length of template_copy
        diff = len(template_copy) - len(predictions_scaled)
        predictions_scaled = np.concatenate([predictions_scaled, np.zeros((diff, predictions_scaled.shape[1]))])
    template_copy.iloc[:, 1:] = predictions_scaled
    template_copy.to_csv(filename, index=False)
    return predictions_scaled

save_predictions(y_pred_cnn_full[:-1], 'Predicted_CNN.csv')
save_predictions(y_pred_lstm_full[:-1], 'Predicted_LSTM.csv')
save_predictions(y_pred_rf_full[:-1], 'Predicted_RF.csv')
hybrid_predictions_full = save_predictions(hybrid_predictions_full[:-1], 'Predicted_Hybrid.csv')


## Evaluate Hybrid Model on AggregatedLoad_A


In [18]:
# Load ground truth data for AggregatedLoad_A
true_A = pd.read_csv("TestData_A.csv", index_col=0)
true_A = true_A.iloc[:, 1:]  # Ignore aggregate load column
true_A_values_scaled = true_A.values / true_A.max(axis=0).values

# Calculate and show summed RMSE for the hybrid model on AggregatedLoad_A
pred_A_values_scaled = hybrid_predictions_full / true_A.max(axis=0).values
rmse_sum_hybrid = np.sum(mean_squared_error(true_A_values_scaled, pred_A_values_scaled, multioutput="raw_values"))
print("Hybrid Model RMSE Sum on AggregatedLoad_A:", rmse_sum_hybrid)


Hybrid Model RMSE Sum on AggregatedLoad_A: 0.5639957008322072
